<a href="https://colab.research.google.com/github/gvogiatzis/CS4740/blob/main/CS4740_Lab_Week_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [CS4740 Labs] Week 5: Exploring the capabilities of Recurrent Neural Networks

## Introduction

In this lab we will carry out a few experiments with Recurrent Neural Networks in order to understand the way these models operate and also to see some interesting applications. RNNs are Deep Learning models that are particularly suitable for sequential data such as time series, text, speech, audio signals etc.

We can think of an RNN as a *machine* that is fed a sequence of data-points. After each data-point is entered, the machine performs some calculations based on the information received, and it  modifies its inner state appropriately. It then produces some output before moving to the next data-point in the sequence. 

We can use RNNs in a multitude of ways:

* We can use the last RNN output after a sequence has been read, to make some inference about the type of sequence. E.g. classifying a bit of text into one of a number of categories.
* We can use an RNN to encode an input sequence, and pass a code to a second RNN that decodes that into an output sequence. This can form the basis of a sequence-to-sequence mapping, e.g. translation of english to french text.
* We can use an RNN as a next-step predictor. E.g. predicting the next day's stock market price from its past history.

In this lab we will see a few examples of RNNs: 

1. A very simple bit-counter
2. A text generator
3. An *adder* that parses numerical expressions

Firstly let's import a few packages:

In [ ]:
import re
import csv
from textblob import Word
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from more_itertools import sliced
from torch.utils.tensorboard import SummaryWriter

from random import randint

%load_ext tensorboard

## A simple running total calculator

This is perhaps the simplest possible RNN we can come up with, that still has a usefull function. The idea is simple: we feed a sequence of numbers and the RNN calculates a running total over those numbers as they come in. 

First, let's see how the rnn layers work in practice. The main two choices are the GRU and the LSTM, details of which we have seen in lectures. We will be using the GRU throughout this lab but both are drop-in replacements for each other.

Let's define a simple GRU layer:

In [ ]:
gru = nn.GRU(input_size=2, hidden_size=5)

This module expects to receive sequences of 2-dimensional data points. It crunches them using a hidden layer of size 5, which means that it outputs a sequence of 5-element vectors. Now the actual dimensionality of the input and output is somewhat confusing because of the complexity of batches: it is very important that all NN layers in pytorch are able to handle batched input for standard stochastic gradient descent to work. 

So the input to RNN layers (by default) is of size 

 `sequence_size` x `batch_size` x `input_size`

Why is `batch_size` in the middle I hear you ask? Well it makes sense if we want to iterate through the elements of the sequence. Each element in that sequence is of size `batch_size` x `input_size` and that's exactly what the inner layers of the RNN expect when operating on batches!

Having said that, if it is more natural for you to think in terms of batches as the rightmost index, as you would do for a conv or linear layer, `pytorch` allows you to do that by using the `batch_first=True` flag inside RNN constructors. In fact we will be making use of that flag throughout this lab, but bear in mind that `pytorch` is flipping the indices internally.

So in our case we would define an input consisting of a batch of 64 sequences, each of which has 100 elements, that are 2-dimensional, we would do it as follows:

In [ ]:
gru = nn.GRU(input_size=2, hidden_size=5)

x = torch.randn(100,64,2)
y,h = gru(x)

y.shape

But if we use the `batch_first=True` flag, this becomes:

In [ ]:
gru = nn.GRU(input_size=2, hidden_size=5, batch_first=True)

x = torch.randn(64,100,2)
y,h = gru(x)

y.shape

Now back to our running total network, the input will be one-dimensional while we can use a relatively small hidden layer of size 64. This is because the task is quite simple so should be easy to handle with this small number of nodes. We will however need to map those 64 nodes to a 1-dimensional output which is the running total. The best way to do that is with a linear layer from 64 to 1 dimension. Alltogether this looks like this:






In [ ]:
class RunningTotalNet(nn.Module):
    def __init__(self, hidden_dim=64):
        super(RunningTotalNet, self).__init__()
        self.rnn = nn.GRU(input_size=1,
                        hidden_size=hidden_dim,
                        batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, batch_size=1):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

Now we can write some training code. We can generate the data randomly for each batch we train on, using the torch `randint` command that will generate for us random integers from 0 to 9. Each batch will be of size 

`(batch_size,sequence_length,1)`

We can also easily obtain the running total output that we want our network to learn, using the `cumsum` command that computes the cummulative summation over a particular tensor dimension.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size=100
sequence_length=50
num_of_epochs = 5000
net = RunningTotalNet().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.MSELoss()
for e in range(num_of_steps):
    x = torch.randint(high=10, (batch_size,sequence_length,1), device=device, dtype=torch.float32)
    t = x.cumsum(dim=1,dtype=torch.float32)
    y = net(x)
    L = loss(y,t)

    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    print(f"\rEpoch: {e}/{num_of_epochs} \tL={L}", end="")

Let's evaluate how well the network has learned:

In [ ]:
def evaluate_net(net, input):
    x = torch.tensor(input, device=device, dtype=torch.float32).view(1,-1,1)
    y = net(x)
    return y.squeeze().tolist()

Trying out a few different sequences

In [ ]:
print(evaluate_net(net,[1,0,0,5]))
print(evaluate_net(net,[1,2,3,4]))

What about negative numbers or bigger than 9?

In [ ]:
print(evaluate_net(net,[-1,-1,0,-2]))
print(evaluate_net(net,[0,20,0,15]))

The network shows signs that it has trully learned to accumulate numbers, beyond just memorizing its input! 

Here's some more detailed code that plots side by side the expected output vs what the network produces, for a randomly picked sequence of numbers. Play with the `low` and `high` parameters to stress the network outside its comfort zone!

> For the curious student:  can you check what happens if we increase the size of the hidden layer? Will the network overfit and stop generalizing?

In [ ]:
import matplotlib.pyplot as plt

x = torch.randint(low=-2, high=20, size=(1,15,1), device=device, dtype=torch.float32)
t = x.cumsum(dim=1,dtype=torch.float32)
t = t.squeeze().tolist()
y = net(x).squeeze().tolist()

plt.plot(t,'r-')
plt.plot(y, 'b+')
plt.show()

## A random text generator

In this section we will use a character-level RNN as a generator of text. The idea is to create a NN that can predict the next character from a string of all the previous ones. Then we can feed the predicted character back to the network so that it can predict the next one, and the next after that etc.

First, we need some more libraries:

In [ ]:
import csv
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from more_itertools import sliced
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

We will use a well known publicly available dataset consisting of news articles from BBC news. The articles each fall under one of five classes, and can be used for document classification. Here we will only be using the raw text of all the news stories, combined.

In [ ]:
! wget https://github.com/suraj-deshmukh/BBC-Dataset-News-Classification/raw/master/dataset/dataset.csv -O dataset.csv

The following snippet opens the csv file and then from every row, takes the "news" field that contans the news story as a string. All these news stories are then concatenated using the `join` command. We will only be using the first million characters, just to keep things manageable in the space of a single lab session. Feel free to experiment with using the full dataset.

In [ ]:
with open('dataset.csv', newline='', encoding = "ISO-8859-1") as csvfile:
    reader = csv.DictReader(csvfile)
    all_text = "".join(row['news'] for row in reader)

all_text = all_text[:1000000]

As is commonly the case in Deep Learning methods for natural language processing, we must first turn individual characters (or words if we have a word-level RNN) into indices (i.e. integer numbers). The best way to do this is using a python dictionary. In fact we will define python dictionaries that map an index to a character (`itoc`) as well as a character to its index (`ctoi`).

In [ ]:
all_chars = sorted(set(all_text))
num_of_characters = len(all_chars)
itoc = {i:c for i,c in enumerate(all_chars)} # this is just for clarity, this dict is same as all_chars 
ctoi = {c:i for i,c in enumerate(all_chars)}

Now we can easily convert a sequence of characters into a sequence of numbers (indices). However as is usually the case with categorical variables (i.e. those that can take one out of a finite number of possible values) they must be converted into vectors (e.g. one-hot encodings) before we can process them further. The reasons are somewhat complex, but fundamentally it's because integer indices have a natural ordering. E.g. index 2 is somehow 'close' to index 3. If we would like our algorithm to take that into account then integer encodings are fine. Usually however we don't want to assume any ordering and in these cases we are obliged to use vector representations. 

In fact we will go one step further than simple one-hot encodings. We will define a general, learnable vector for each of the characters. This is easily achieved using what is known as an embedding layer.

Consider just for illustration's sake a very simple 4-dimensional embedding that maps from a character index to a 4D vector. This is defined as follows:

In [ ]:
embedding = nn.Embedding(num_of_characters, 4)

This is a structure that produces a different 4-d vector for each of the different characters in our character set.

In [ ]:
embedding(torch.tensor(ctoi['s']))

Let's apply it to a longer string:

In [ ]:
# a string variable
txt = 'hello' 

# converting it into an array of indices
x = [ctoi[c] for c in txt]

# converting into a tensor
x = torch.tensor(x)

y = embedding(x)

print(y.shape)

Getting the dimensions right in `pytorch` is always tricky. That's why it's important to try out the layers with toy input data, to make sure it all fits together as expected. 

We are now ready to define a network that will predict the next character given some text. This is essentially a classification task. Because we want our network to be able to use input sequences of variable length we will use a recurrent network in the middle of the computation. Here we chose a GRU, but an LSTM will also work.

The structure of the predictor network is as follows:

input char sequence -> embedding -> GRU -> fully connected layer -> softmax

As we will be using a `CrossEntropyLoss` loss function, we can omit the softmax layer in the end. The code looks like this:

In [ ]:
class CharPredictorNet(nn.Module):
    def __init__(self, charset_size, embed_size=100, hidden_dim=1024):
        super(CharPredictorNet, self).__init__()
        self.embedding = nn.Embedding(charset_size, embed_size)
        self.lstm = nn.GRU(input_size=embed_size,
                            hidden_size=hidden_dim,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, charset_size)
        self.h=None

    def forward(self, x, keep_memory=False):
        x = self.embedding(x)
        if keep_memory: # if keep_memory, we use the memory vector from previous run
            x, self.h = self.lstm(x,self.h) 
        else:           # else we start a new run (memory is initialized to zero)
            x, self.h = self.lstm(x) 
        x = self.fc(x)
        return x

Testing that it all works in terms of the dimensions:

In [ ]:
x = torch.tensor([1,2,0,2,50,20])
net = NextCharPredictor(charset_size=100)
y = net(x.view(1,-1))
print(y.shape)

Remeber that RNNs in pytorch can handle mini-batches. If you use the `batch_first=True` flag, then the networ expects inputs of shape:

batch_size x sequence_size x element_size


where batch_size is the number of sequences contained in your batch, sequence_size is the length of each sequence and element_size is the size of the vector that represents each sequence element (in our case a character embedding vector)

It will come handy to define a simple running average class to keep track of the average accuracy accross an epoch. This can be done as follows:

In [ ]:
class RunningAverage:
    def __init__(self):
        self.n=0
        self.tot=0
    
    def add(self,x):
        self.n += 1
        self.tot += x
        
    def __call__(self):
        return self.tot/self.n

As before, we will start up a tensorbord GUI to monitor our experiments.

In [ ]:
tensorboard --logdir=runs

We are now ready to write the training code. In each epoch we will be using a chunk of the whole text that is stored in the `all_text` variable. This chunking is done by the `sliced` command that can be found in the `more_itertools` package. Each chunk will have length 

`batch_size*seq_length+1`

and will be used to form `batch_size` sequences, each of length `seq_length`. We can use the same string to generate both inputs and expected outputs by exploiting the fact that the output is just the same as the input but shifted by one position to the right. I.e. the network has to predict the next character, having seens all characters up to, but not including that charracter. 

We use a neat little python idiom that does the shifting. If variable `txt` holds string `'hello world'` then the two variables `txt[:-1]` and `txt[1:]` hold 

`'hello worl'`

and

`'ello world'`

respectively. Which means that we can use `txt[:-1]` as the input and `txt[1:]` as the target. If our chunk `txt` has length 

`batch_size*seq_length+1`

then `txt[:-1]` and `txt[1:]` each have length 

`batch_size*seq_length`

so we will be able to reshape them into a `batch_size` x `seq_length` tensor for further processing inside the network.

The training code looks as follows:

In [ ]:
num_of_epochs = 50
seq_length = 100
batch_size=64*2
embed_size=100
hidden_dim=1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = CharPredictorNet(embed_size=embed_size, hidden_dim=hidden_dim, charset_size = num_of_characters).to(device)
loss = nn.CrossEntropyLoss()
optim = torch.optim.Adam(net.parameters(), lr=0.001) 

max_iter = int(len(all_text)/(batch_size*seq_length+1))

writer = SummaryWriter(f'runs/exp_1M_rnd_batch128')

for e in range(num_of_epochs):
    train_acc = RunningAverage()
    for i,txt in enumerate(sliced(all_text, batch_size*seq_length+1)):
        if len(txt)<batch_size*seq_length+1:
            break
        x = torch.tensor([ctoi[c] for c in txt[:-1]], device = device)
        t = torch.tensor([ctoi[c] for c in  txt[1:]], device = device)        
        y = net(x.view(batch_size,-1)) # we must wrap so it can be processed as a batch
        y = y.view(-1,num_of_characters) # we unwrap y to compare it with the unwrapped target
        L = loss(y, t)
        acc = sum(y.argmax(dim=1)==t).item()/(batch_size*seq_length)
        train_acc.add(acc)
        print(f"\rEpoch: {e}/{num_of_epochs} Iter: {i}/{max_iter}\tacc={100*acc:0.2f}%\tL={L}", end="")
        optim.zero_grad()
        L.backward()     
        optim.step()
    writer.add_scalar('Accuracy', train_acc(), e)
    writer.flush()
    print(f"\rEpoch: {e}/{num_of_epochs} Average acc: {train_acc()}")
writer.close()

The following is a code snippet that runs the network in a generative way. Taking its output and feeding it back in as input we can keep on generating text for arbitrary lengths. 

In [ ]:
def generate_text(net, seed_txt, length=100):
    indices = [ctoi[c] for c in seed_txt]
    x = torch.tensor(indices, device = device)
    
    x = net(x) # compute network predicted probabilities for next char
    x = x.argmax(dim=1).view(-1) # get most probable char according to network
    x = x[-1] # take the last character in the output to use as input
    indices.append(x.item()) #add to text 
    for i in range(length):
        x=net(x, keep_memory=True) # compute network predicted probabilities for next char
        x = x.argmax(dim=1).view(-1)# get most probable char according to network
        indices.append(x.item()) #add to text
    return  "".join(itoc[i] for i in indices) # convert back to string


Try out a few seed texts to see how the network completes them 

In [ ]:
print(generate_text(net, "chine", length=200))



> For the curious: Can you try the algorithm on a different dataset? E.g. you might want to use some of the programming language code you have written for other modules. Try to see if the network learns how to write passable python/java programs! 



## An interpreter of numerical expressions

We will now look at a slightly more challenging problem, that of parsing and interpreting numerical expresions. This may sound like a simple problem but some consideration will convince you it is non-trivial to solve using a general-purpose computational device such as a neural network. 

As an example, our network will be fed a sequence such as 

`'125+238='` 

and it will be able to respond with the correct answer which is

`'363.'`

For the purposes of this lab we will restrict the problem to just two integers between 0 and 999 and they will only be combined using additions. Firstly, we must be able to automatically generate training data. This is done with the following two functions.

In [ ]:
import csv
import numpy as np

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from more_itertools import sliced
from torch.utils.tensorboard import SummaryWriter

itoc=list('0123456789+=. ')
ctoi = {c:i for i,c in enumerate(itoc)}

def create_strings(a,b):
    c=a+b
    a_s=str(a)
    b_s=str(b)
    c_s=str(c)
    input = f'{a_s}+{b_s}={c_s}'
    output = f'{c_s}.'
    input = f'{a_s}+{b_s}'
    output=' ' * len(input)
    input += f'={c_s}'
    output+= f'{c_s}.'

    
    return input, output

def create_addition_samples(num_samples,ndigits):
    x,t=[],[]
    for n in range(num_samples):
        a = randint(0,10**ndigits-1)
        b = randint(0,10**ndigits-1)
        input, output = create_strings(a,b)
        input = input.ljust(3*ndigits+3, ' ')
        output = output.ljust(3*ndigits+3, ' ')
        x.append([ctoi[c] for c in input])
        t.append([ctoi[c] for c in output])
    return x,t

Take a while to study how these functions work. Also generate a few strings for some numbers. E.g.

In [ ]:
x,y = create_strings(150,82)
print(x)
print(y)

The idea works very much like the next-character predictor that we created above. In fact the neural network architecture is identical, we don't need a new class. 

The input sequence is the arithmetic expression and the output is a series of space characters, until the network encounters the `'='` character. At that point the network is required to start outputing the result of the addition. This is very similar to the offsetting of input and output by one character that we saw before. The end effect is that, if the network manages to learn this, as soon as it encounters `'='`, it will perform the calculation and output the result. 

The same effect could probably be achieved with a seq2seq architecture with an encoder and a decoder RNN. That would perhaps be a more general solution. However this problem is simple enough that a single network can play both roles.

The training code is very straightforward:

In [ ]:
from random import randint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size=100
num_of_epochs = 2000
ndigits=3
net = CharPredictorNet(charset_size = len(itoc),embed_size=32, hidden_dim=2048).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()
for e in range(num_of_epochs):
    x,t = create_addition_samples(batch_size,ndigits=ndigits)
    x = torch.tensor(x,device=device)
    t = torch.tensor(t,device=device)
    y = net(x)
    L = loss(y.view(-1,len(itoc)),t.view(-1))
    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    print(f"\rEpoch: {e}/{num_of_epochs} \tL={L}", end="")

We now write a simple function, similar to the one for the text generator, to evaluate the numerical expression interpreter network. Instead of priming the RNN with a bit of text we would like to continue, we prime it with the mathematical expression we would like to interpret.

In [ ]:
def eval_adder(net, input_str):
    itoc=list('0123456789+=. ')
    ctoi = {c:i for i,c in enumerate(itoc)}
    input = torch.tensor([[ctoi[c] for c in input_str]], device = device)
    x = net(input).argmax(dim=2)
    x = x[0,-1].view(1,-1)
    output = [x.item()]
    total_len=0
    while x.item() != ctoi['.'] and total_len<10:
        x = net(x, keep_memory=True).argmax(dim=2)
        output.append(x.item())
        total_len+=1
    return "".join(itoc[i] for i in output)

Let's test it out!

In [ ]:
eval_adder(net, "555+234=")

In [ ]:
eval_adder(net, "999+999=")

It seems to work! 



> For those of you who would like to investigate further:

* Can you find some cases where the interpreter fails? 
* What about examples that have not been shown to the network?
* Can you extend this idea to a fully fledged interpreter that covers `+,-,*,/`? Maybe even parentheses? Have fun!

